# Fase 1: Limpeza e Pré-processamento dos Dados

Este notebook executa a primeira fase do projeto de previsão de churn: a limpeza e o pré-processamento dos dados. As seguintes etapas serão realizadas:

1.  **Carregamento dos Dados**: Importação das bibliotecas e leitura do arquivo CSV.
2.  **Tratamento da Variável Alvo (`Churn`)**: Remoção de registros onde a variável alvo é nula.
3.  **Correção de Tipos e Tratamento de Nulos (`Charges.Total`)**: Conversão da coluna para o tipo numérico e tratamento inteligente dos valores ausentes.
4.  **Validação da Consistência dos Dados**: Verificação de regras de negócio entre as colunas de serviços.
5.  **Salvamento dos Dados Limpos**: Exportação do DataFrame processado para um novo arquivo CSV.

In [ ]:
import pandas as pd
import numpy as np

# Desabilitar warnings para uma saída mais limpa (opcional)
import warnings
warnings.filterwarnings('ignore')

### 1. Carregamento dos Dados

In [ ]:
# O caminho do arquivo foi fornecido no contexto.
caminho_arquivo = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x.csv'

# Carregar o dataset
df = pd.read_csv(caminho_arquivo)

# Exibir as primeiras linhas e informações gerais
print("Dimensões Originais do Dataset:", df.shape)
print("\nInformações Iniciais:")
df.info()

### 2. Tratamento da Variável Alvo (`Churn`)

A variável alvo é essencial para o treinamento de um modelo supervisionado. Registros com o valor de `Churn` ausente não podem ser utilizados e serão removidos.

In [ ]:
print(f"Quantidade de valores nulos em 'Churn' antes: {df['Churn'].isnull().sum()}")

# Remover linhas onde a coluna 'Churn' é nula
df.dropna(subset=['Churn'], inplace=True)

print(f"Quantidade de valores nulos em 'Churn' depois: {df['Churn'].isnull().sum()}")
print(f"Novas dimensões do Dataset: {df.shape}")

### 3. Correção de Tipos e Tratamento de Nulos (`Charges.Total`)

A coluna `Charges.Total` frequentemente é carregada como `object` (texto) se contiver espaços ou outros caracteres não numéricos. O primeiro passo é convertê-la para um formato numérico, forçando os valores inválidos a se tornarem `NaN` (Not a Number).

In [ ]:
# Converter 'Charges.Total' para numérico. errors='coerce' transforma o que não for número em NaN.
df['Charges.Total'] = pd.to_numeric(df['Charges.Total'], errors='coerce')

# Verificar quantos valores nulos foram criados/encontrados
nulos_total_charges = df['Charges.Total'].isnull().sum()
print(f"Quantidade de valores nulos em 'Charges.Total' após conversão: {nulos_total_charges}")

#### 3.1 Investigando a Causa dos Valores Nulos

Antes de aplicar uma regra genérica de imputação, vamos investigar por que esses valores são nulos. A hipótese é que eles pertencem a clientes novos com `tenure` (tempo de contrato) igual a 0.

In [ ]:
# Filtrar o dataframe para ver os registros onde 'Charges.Total' é nulo
registros_nulos = df[df['Charges.Total'].isnull()]

print("Valores de 'tenure' para registros com 'Charges.Total' nulo:")
print(registros_nulos['tenure'].value_counts())

#### 3.2 Imputação dos Valores Nulos

A investigação confirma que todos os valores ausentes em `Charges.Total` correspondem a clientes com `tenure = 0`. Portanto, a imputação correta e que reflete a realidade do negócio é preencher esses valores com `0`.

**Nota sobre a Regra de Imputação:** Embora a instrução geral fosse usar amostragem bootstrap para um grande número de nulos, neste caso específico, a investigação revelou uma causa raiz definida. A imputação com '0' é uma representação factual e mais precisa do que uma imputação estatística aleatória.

In [ ]:
# Preencher os valores nulos em 'Charges.Total' com 0
df['Charges.Total'].fillna(0, inplace=True)

print(f"Valores nulos em 'Charges.Total' após imputação: {df['Charges.Total'].isnull().sum()}")

### 4. Validação da Consistência dos Dados

Verificamos se há consistência entre os serviços de internet. Por exemplo, se `InternetService` for 'No', então colunas como `OnlineSecurity`, `TechSupport`, etc., devem ter o valor 'No internet service'.

In [ ]:
colunas_servicos_internet = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Filtra o dataframe onde não há serviço de internet
df_sem_internet = df[df['InternetService'] == 'No']

print("Verificando a consistência para clientes sem internet:\n")
for col in colunas_servicos_internet:
    # Se todos os valores forem 'No internet service', o output será 1. Qualquer outro valor indica inconsistência.
    is_consistent = df_sem_internet[col].nunique() == 1 and df_sem_internet[col].unique()[0] == 'No internet service'
    print(f"{col}: {'Consistente' if is_consistent else 'INCONSISTENTE'}")

### 5. Salvamento dos Dados Limpos

Após a limpeza e o pré-processamento, o DataFrame resultante é salvo em um novo arquivo CSV para ser usado nas próximas fases de análise e modelagem.

In [ ]:
caminho_saida = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_cleaned.csv'

df.to_csv(caminho_saida, index=False)

print(f"\nDataset limpo e salvo com sucesso em: {caminho_saida}")
print(f"Dimensões finais do Dataset: {df.shape}")